### Selected Word Group In Youtube Sentence

This notebook is used to find youtube videos for words and phrases.

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Turkish_200_Word"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "200_Word"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 6  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [10]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [11]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [12]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,4832.485,4835.625,yani can bey siz demek istiyorsunuz ki,lrLbXImf8rk
1,1216.637,1218.362,hayat ne garip değil mi ali,wsWnhczse10
2,4803.325,4805.295,adamı vedat görüşme talep ediyor,de9wmUKoZxI
3,119.552,125.696,yumuşatıp özleştiriyorum birkaç dakika sonra s...,wL9OiUePql8
4,786.386,787.186,nefise,9J7s6l7Ob4Q
...,...,...,...,...
3934202,5271.620,5274.964,abdullah abi venedike gitti ney,nRbLFo7bTaQ
3934203,4716.925,4720.328,ve i slama aykırı tek bir işi yoktur,6teAhXTWpsU
3934204,6678.687,6679.772,ulan,x4ioRas_9l8
3934205,173.329,181.240,bu sadece genel satılan bir ürün olduğu için ç...,bm4w8nD3JzQ


In [13]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [ ]:
# Option 
# Skip if it doesn't need
df_remove = pd.read_excel(f"")
df_remove

In [ ]:
remove_videoid_list = list(set(df_remove["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [14]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [15]:
len(search_list)

200

In [16]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,istiyorum,16.699,18.230,istiyorum dedi,ew7gnKjLjQc
1,istiyorum,6914.376,6918.336,fikret gelmeni ve yanımda olmanı istiyorum lütfen,zmmMNX_dDg4
2,istiyorum,2247.944,2250.663,ben somut bir bilim istiyorum bir kanıt istiyo...,Q3Azp5eUgNE
3,istiyorum,6328.971,6331.362,baba lütfen kurt derisi istiyorum,iMj5gB0XHRw
4,istiyorum,7439.975,7442.410,ama ben bir şeyi hatırlatmak istiyorum yine de,W2LJG6d5398
...,...,...,...,...,...
1195,uzun,2929.919,2936.063,uzun zamandır yapmam gerekeni yapıyorum şimdi ...,CIQ3Ave21Kw
1196,uzun,446.463,449.535,çok uzun süreler fermente olmasını beklemeyece...,C0YofWzdFmM
1197,uzun,4573.260,4580.740,uzun zamandır görmedim ama madem herkesin çıka...,vMxEQFO8teQ
1198,uzun,4551.480,4555.200,ama anlaşılıyor ki çok uzun yaşayacaksın,JGuGk584FW8


In [17]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,istiyorum,16.699000,17.792571,istiyorum dedi,ew7gnKjLjQc
1,istiyorum,6916.962122,6917.851102,fikret gelmeni ve yanımda olmanı istiyorum lütfen,zmmMNX_dDg4
2,istiyorum,2248.866518,2249.400607,ben somut bir bilim istiyorum bir kanıt istiyo...,Q3Azp5eUgNE
3,istiyorum,6330.637455,6331.362000,baba lütfen kurt derisi istiyorum,iMj5gB0XHRw
4,istiyorum,7441.457174,7442.039457,ama ben bir şeyi hatırlatmak istiyorum yine de,W2LJG6d5398
...,...,...,...,...,...
1195,uzun,2929.919000,2930.317961,uzun zamandır yapmam gerekeni yapıyorum şimdi ...,CIQ3Ave21Kw
1196,uzun,446.640231,446.994692,çok uzun süreler fermente olmasını beklemeyece...,C0YofWzdFmM
1197,uzun,4573.260000,4573.733418,uzun zamandır görmedim ama madem herkesin çıka...,vMxEQFO8teQ
1198,uzun,4553.526000,4554.084000,ama anlaşılıyor ki çok uzun yaşayacaksın,JGuGk584FW8


In [18]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,istiyorum,16.099000,18.392571,istiyorum dedi,ew7gnKjLjQc
1,istiyorum,6916.362122,6918.451102,fikret gelmeni ve yanımda olmanı istiyorum lütfen,zmmMNX_dDg4
2,istiyorum,2248.266518,2250.000607,ben somut bir bilim istiyorum bir kanıt istiyo...,Q3Azp5eUgNE
3,istiyorum,6330.037455,6331.962000,baba lütfen kurt derisi istiyorum,iMj5gB0XHRw
4,istiyorum,7440.857174,7442.639457,ama ben bir şeyi hatırlatmak istiyorum yine de,W2LJG6d5398
...,...,...,...,...,...
1195,uzun,2929.319000,2930.917961,uzun zamandır yapmam gerekeni yapıyorum şimdi ...,CIQ3Ave21Kw
1196,uzun,446.040231,447.594692,çok uzun süreler fermente olmasını beklemeyece...,C0YofWzdFmM
1197,uzun,4572.660000,4574.333418,uzun zamandır görmedim ama madem herkesin çıka...,vMxEQFO8teQ
1198,uzun,4552.926000,4554.684000,ama anlaşılıyor ki çok uzun yaşayacaksın,JGuGk584FW8


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,istiyorum,16,18,istiyorum dedi,ew7gnKjLjQc
1,istiyorum,6916,6918,fikret gelmeni ve yanımda olmanı istiyorum lütfen,zmmMNX_dDg4
2,istiyorum,2248,2250,ben somut bir bilim istiyorum bir kanıt istiyo...,Q3Azp5eUgNE
3,istiyorum,6330,6332,baba lütfen kurt derisi istiyorum,iMj5gB0XHRw
4,istiyorum,7441,7443,ama ben bir şeyi hatırlatmak istiyorum yine de,W2LJG6d5398
...,...,...,...,...,...
1195,uzun,2929,2931,uzun zamandır yapmam gerekeni yapıyorum şimdi ...,CIQ3Ave21Kw
1196,uzun,446,448,çok uzun süreler fermente olmasını beklemeyece...,C0YofWzdFmM
1197,uzun,4573,4574,uzun zamandır görmedim ama madem herkesin çıka...,vMxEQFO8teQ
1198,uzun,4553,4555,ama anlaşılıyor ki çok uzun yaşayacaksın,JGuGk584FW8


In [20]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,istiyorum,16,18,istiyorum dedi,ew7gnKjLjQc,https://www.youtube.com/watch?v=ew7gnKjLjQc&t=16s
1,istiyorum,6916,6918,fikret gelmeni ve yanımda olmanı istiyorum lütfen,zmmMNX_dDg4,https://www.youtube.com/watch?v=zmmMNX_dDg4&t=...
2,istiyorum,2248,2250,ben somut bir bilim istiyorum bir kanıt istiyo...,Q3Azp5eUgNE,https://www.youtube.com/watch?v=Q3Azp5eUgNE&t=...
3,istiyorum,6330,6332,baba lütfen kurt derisi istiyorum,iMj5gB0XHRw,https://www.youtube.com/watch?v=iMj5gB0XHRw&t=...
4,istiyorum,7441,7443,ama ben bir şeyi hatırlatmak istiyorum yine de,W2LJG6d5398,https://www.youtube.com/watch?v=W2LJG6d5398&t=...
...,...,...,...,...,...,...
1195,uzun,2929,2931,uzun zamandır yapmam gerekeni yapıyorum şimdi ...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
1196,uzun,446,448,çok uzun süreler fermente olmasını beklemeyece...,C0YofWzdFmM,https://www.youtube.com/watch?v=C0YofWzdFmM&t=...
1197,uzun,4573,4574,uzun zamandır görmedim ama madem herkesin çıka...,vMxEQFO8teQ,https://www.youtube.com/watch?v=vMxEQFO8teQ&t=...
1198,uzun,4553,4555,ama anlaşılıyor ki çok uzun yaşayacaksın,JGuGk584FW8,https://www.youtube.com/watch?v=JGuGk584FW8&t=...


In [21]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

200

In [22]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [23]:
search_all_set.difference(search_result_set)

set()

In [24]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,adam,1
1,bana,1
2,bilmiyorum,1
3,ama,1
4,anne,1
...,...,...
195,şey,1
196,şeyi,1
197,şeyler,1
198,şimdi,1


In [25]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [26]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [27]:
set_word.difference(set_count)

set()

In [28]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass